<a href="https://colab.research.google.com/github/srikarraju/GridWorld/blob/main/Natural_Actor_Critic_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import gym
from collections import deque
import numpy as np

In [ ]:
def generate_state_action_features(state_vec, d,num_actions):
  state_action_features = np.zeros(shape=(num_actions,d+num_actions))
  for i in range(num_actions):
    for j in range(d):
      state_action_features[i][j] = state_vec[j]
  state_action_features[0][d] = 1
  state_action_features[1][d+1] =  1
  return state_action_features

In [ ]:
env = gym.make('CartPole-v0')

returns = deque(maxlen=100)
weights_v = np.zeros(4,dtype=float)
weights_p = np.zeros(6,dtype=float)
print(weights_v)
print(weights_p)

[0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]


In [ ]:
alpha_0, beta_0, gamma, epsilon = 0.1, 0.01, 0.95, 0.1
alpha_c, beta_c = 1000, 1000
t = 0
n_episode = 1
avg_reward = 0
actions_list = []
fischer_inv = 1.5*np.eye(6)

alpha = 0.00001
beta = 0.000001

while n_episode <=10000:
  rewards,states,actions = [],[],[]
  state = env.reset()
  t = 0

  while True:
    t += 1
    state_action_features =generate_state_action_features(np.asarray(state),4,2)

    probs = np.dot(state_action_features,weights_p)
    probs -= probs.max()
    probs = np.exp(np.clip(probs/epsilon, -500, 500))
    probs /= probs.sum()

    probs2 = probs.cumsum()
    action = np.where(probs2 >= np.random.random())[0][0]

    new_state, reward, done, info = env.step(action.item())


    value_curr = np.dot(weights_v,np.asarray(state))
    value_next = np.dot(weights_v,np.asarray(new_state))

    beta = (beta_0 * beta_c) / (beta_c + t)
    alpha = (alpha_0 * alpha_c) / (alpha_c + t**(2/3))

    avg_reward = (1 - alpha*gamma)*avg_reward + alpha*gamma * reward
    td_error = reward + value_next - value_curr - avg_reward

    weights_v += alpha * td_error * np.asarray(state)/np.amax(state)

    grad_prob = probs[action]*(1-probs[action])*state_action_features[action]/np.amax(state_action_features[action])

    Gt_psi_dot = np.dot(fischer_inv, grad_prob)
    fischer_inv -= (0.001*alpha * np.outer(Gt_psi_dot,np.transpose(Gt_psi_dot)) ) / (1 - 0.001*alpha + 0.001*alpha * np.dot(grad_prob,Gt_psi_dot))
    fischer_inv /= (1 - 0.001*alpha)

    weights_p += beta * td_error * np.dot(fischer_inv,grad_prob)

    states.append(state)
    actions.append(action)
    rewards.append(reward)
    actions_list.append(action)

    state = new_state
    if done==True:
      break

  returns.append(np.sum(rewards))
  if n_episode%100==0:
    print("Episode: {:6d}\tAvg. Return: {:6.2f}".format(n_episode, np.mean(returns)))
  n_episode += 1

env.close()

Episode:    100	Avg. Return:  11.86
Episode:    200	Avg. Return:   9.34
Episode:    300	Avg. Return:   9.34
Episode:    400	Avg. Return:   9.49
Episode:    500	Avg. Return:   9.52
Episode:    600	Avg. Return:   9.35
Episode:    700	Avg. Return:   9.41
Episode:    800	Avg. Return:   9.25
Episode:    900	Avg. Return:   9.39
Episode:   1000	Avg. Return:   9.40
Episode:   1100	Avg. Return:   9.26
Episode:   1200	Avg. Return:   9.36
Episode:   1300	Avg. Return:   9.23


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


IndexError: ignored

In [ ]:
print(weights_p)

[nan nan nan nan nan nan]


In [ ]:
print(np.outer(Gt_psi_dot,np.transpose(Gt_psi_dot)))

[[ 4.06892924e+33 -4.42195146e+32  4.18739412e+33 -7.34078041e+32
  -2.87071525e+17]
 [-4.42195146e+32  4.80560206e+31 -4.55069441e+32  7.97766999e+31
   3.11977986e+16]
 [ 4.18739412e+33 -4.55069441e+32  4.30930805e+33 -7.55450364e+32
  -2.95429471e+17]
 [-7.34078041e+32  7.97766999e+31 -7.55450364e+32  1.32435474e+32
   5.17907514e+16]
 [-2.87071525e+17  3.11977986e+16 -2.95429471e+17  5.17907514e+16
   2.02535005e+01]]
